## シェープファイル または dBASE ファイル の LDID（Language driver ID）と *.cpg ファイル を調べる

### 概要：

「[操作手順: 各種コード ページでエンコードされたシェープファイルと dBASE ファイルを読み書きする](https://support.esri.com/ja/technical-article/000013192) 」 の技術文章で記載しているように、ArcGIS の
dBASE ファイルのコード ページ変換機能 (名称: `dbfDefault`) は、システム レジストリにコード ページ値を指定してアクティブ化するものです。ESRIジャパンで提供している、[シェープファイル文字コード設定ユーティリティ](https://doc.esrij.com/pro/get-started/setup/user/addin_tool/#%E3%82%B7%E3%82%A7%E3%83%BC%E3%83%97%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E6%96%87%E5%AD%97%E3%82%B3%E3%83%BC%E3%83%89%E8%A8%AD%E5%AE%9A%E3%83%A6%E3%83%BC%E3%83%86%E3%82%A3%E3%83%AA%E3%83%86%E3%82%A3) は、この`dbfDefault` の設定をユーザーインターフェイス で設定するためのツールです。

また、上記の[操作手順](https://support.esri.com/ja/technical-article/000013192) には、dbfDefault をレジストリに設定することにより出来ることが解説されている他、dBASE ファイル の LDID（Language driver ID）, *.cpgファイル の有無による、アプリケーションでの挙動について次のように解説されています。

「シェープファイルまたは dBASE ファイルに LDID や *.cpg ファイルが存在しない場合、ArcGIS は、dbfDefault で定義されたコード ページでファイルがエンコードされると見なします。 たとえば、dbfDefault 値が OEM に設定され、dBASE ファイルに LDID および *.cpg ファイルが両方とも存在しない場合、ArcGIS はファイルが OEM でエンコードされると見なすため、ArcMap、ArcCatalog、および ArcGIS Pro で 8 ビット文字を表示するためにコード ページ変換を実行します (これは、どのアプリケーションも、ANSI コード ページを使用して文字列を表示する Windows プログラムであるためです)。」

裏を返せば、国内での一部のシェープファイルが文字化けするのは、

- dBASE ファイルの LDID（Language driver ID）が存在しない（値が0）
- オプションの *.cpg ファイル が提供されていない

の両方があてはまる場合であるとも言えます。


**※** 上記の[操作手順](https://support.esri.com/ja/technical-article/000013192) は、デスクトップ製品に関する記載となっていますが、ArcGIS Online や ArcGIS Enterprise でも同様の挙動となります。具体的には、シェープファイル の登録時に指定する ZIP ファイル内に格納された dBASE ファイルの LDID と *.cpg ファイル の両方が存在しない場合、公開後の[フィーチャ レイヤー](https://doc.arcgis.com/ja/arcgis-online/reference/feature-layers.htm) が文字化けした状態になります。

### サンプル ノートブックの目的：

オープンデータ として公開されているシェープファイル（特に作成年が古いもの）や、その他、世の中に流通しているシェープファイルの中には、文字化けが発生する状態で提供されている場合が多々あります。

本ノートブックは、そんな時の原因を理解するために、dBASE ファイル の LDID（Language driver ID）と *.cpg ファイル を、Pure Python （ArcGIS で利用するライブラリは使っていません）で読み込みし確認するためのサンプル ノートブックです。

また、併せてフォルダー内にあるシェープファイルのファイル名にあわせて、*.cpg ファイル を一括で作成するサンプルも記載しております

1. サンプルその１：フォルダー 内のdbf ファイル の LDID と CPGファイルの情報を一覧で表示
2. サンプルその２：サブフォルダー 内のdbf ファイル の LDID と CPGファイルの情報を一覧で表示
3. サンプルその３：フォルダー 内のシェープファイル に対応する CPGファイルを一括で作成する
4. サンプルその４：サブフォルダー 内のシェープファイル に対応する CPGファイルを一括で作成する

なお、シェープファイル作成時の文字コードを UTF-8 で利用する場合の注意点などは、[シェープファイルの文字コードに関する注意](https://esrij-esri-support.custhelp.com/app/answers/detail/a_id/5411)  をご参照ください。

その他、*.cpg ファイル の手動での作成方法は、[問題：シェープファイルや DBF ファイルが文字化けする](https://esrij-esri-support.custhelp.com/app/answers/detail/a_id/5850)  に記載がありますので、そちらをご参照ください。  

※ LDID（Language driver ID）は『 [シェープファイルの技術情報](http://www.esrij.com/cgi-bin/wp/wp-content/uploads/documents/shapefile_j.pdf) 』の中では、「言語ドライバ ID」として記載されているものです。

### ここからがサンプル ノートブックの実装：

LDID がどの Codepage に対応しているか返却できるように定義

In [1]:
# dbaseヘッダーのLDID がどのCodepageに対応しているか
# 取り出しやすくするために、「Shapefile C Library のLanguage driver ID（LDID）と Codepage の対応表」をもとにディクショナリとして定義
# ID(Language driver ID), Codepage, Description
ldid_dic = {
    1:['437','US MS-DOS'],
    2:['850','International MS-DOS'],
    3:['1252','Windows ANSI Latin I'],
    4:['10000','Standard Macintosh'],
    8:['865','Danish OEM'],
    9:['437','Dutch OEM'],
    10:['850','Dutch OEM'],
    11:['437','Finnish OEM'],
    13:['437','French OEM'],
    14:['850','French OEM'],
    15:['437','German OEM'],
    16:['850','German OEM'],
    17:['437','Italian OEM'],
    18:['850','Italian OEM'],
    19:['932','Japanese Shift-JIS'],
    20:['850','Spanish OEM'],
    21:['437','Swedish OEM'],
    22:['850','Swedish OEM'],
    23:['865','Norwegian OEM'],
    24:['437','Spanish OEM'],
    25:['437','English OEM (Great Britain)'],
    26:['850','English OEM (Great Britain)'],
    27:['437','English OEM (US)'],
    28:['863','French OEM (Canada)'],
    29:['850','French OEM'],
    31:['852','Czech OEM'],
    34:['852','Hungarian OEM'],
    35:['852','Polish OEM'],
    36:['860','Portuguese OEM'],
    37:['850','Portuguese OEM'],
    38:['866','Russian OEM'],
    55:['850','English OEM (US)'],
    64:['852','Romanian OEM'],
    77:['936','Chinese GBK (PRC)'],
    78:['949','Korean (ANSI/OEM)'],
    79:['950','Chinese Big5 (Taiwan)'],
    80:['874','Thai (ANSI/OEM)'],
    87:['Current ANSI CP','ANSI'],
    88:['1252','Western European ANSI'],
    89:['1252','Spanish ANSI'],
    100:['852','Eastern European MS-DOS'],
    101:['866','Russian MS-DOS'],
    102:['865','Nordic MS-DOS'],
    103:['861','Icelandic MS-DOS'],
    104:['895','Kamenicky (Czech) MS-DOS'],
    105:['620','Mazovia (Polish) MS-DOS'],
    106:['737','Greek MS-DOS (437G)'],
    107:['857','Turkish MS-DOS'],
    108:['863','French-Canadian MS-DOS'],
    120:['950','Taiwan Big 5'],
    121:['949','Hangul (Wansung)'],
    122:['936','PRC GBK'],
    123:['932','Japanese Shift-JIS'],
    124:['874','Thai Windows/MS–DOS'],
    134:['737','Greek OEM'],
    135:['852','Slovenian OEM'],
    136:['857','Turkish OEM'],
    150:['10007','Russian Macintosh'],
    151:['10029','Eastern European Macintosh'],
    152:['10006','Greek Macintosh'],
    200:['1250','Eastern European Windows'],
    201:['1251','Russian Windows'],
    202:['1254','Turkish Windows'],
    203:['1253','Greek Windows'],
    204:['1257','Baltic Windows']
}

# 補助関数
def codepage_list_from_ldid(ldid):
    """
    LDID をもとに上記に整理したCodepage と Description を含めてリストとして返却
    """
    id_dict = ldid_dic.get(ldid)
    if id_dict == None:
        return [ldid, '0', 'Unknown CodePage']
    else:
        return [ldid, id_dict[0], id_dict[1]]

def codepage_from_ldid(ldid):
    """
    LDID をもとに上記に整理したCodepage と Description を含めて文字として返却
    """
    id_list = codepage_list_from_ldid(ldid)
    return "Language driver ID: {} = CodePage: {} , Description: {}".format(id_list[0], id_list[1],  id_list[2])

# 追加
from struct import *
def codepage_list_from_dbf_file(dbf):
    """
    dBase ファイルのヘッダーからLDIDを取得し、対応するCodepage の情報とともに返却
    """
    with open(dbf,'rb') as f:
        dat = f.read()
    ldid = unpack_from('<B', dat, 29)[0] # ヘッダーからLDID を取得
    return codepage_list_from_ldid(ldid) # LDID をもとにCodepage の情報を取得

*.cpg ファイルの存在の有無と、ある場合には中身の値も返却する定義

In [2]:
import os
def check_cpgfile(folder, cpg_file_name):
    """
    cpg ファイルの存在チェックの関数
    （cpg ファイルが存在する場合はファイルの中身も一緒に取得して返す）
    """
    cpg_file = os.path.join(folder, cpg_file_name)
    bl_exist = os.path.isfile(cpg_file)
    code_value = ""
    if (bl_exist):
        with open(cpg_file, 'r') as f:
            code_value = f.readline().rstrip()
    return (bl_exist, code_value)

def check_related_cpgfile(dbf):
    path, dbf_name = os.path.split(dbf) # path と ファイル名を分ける  
    file_name = os.path.splitext(dbf_name)[0]
    cpg_file_name = "{}.cpg".format(os.path.splitext(file_name)[0])
    cpg_file = os.path.join(path, cpg_file_name)
    bl_exist = os.path.isfile(cpg_file)
    code_value = ""
    if (bl_exist):
        with open(cpg_file, 'r') as f:
            code_value = f.readline().rstrip()
    return (bl_exist, code_value)

#### 1) サンプルその１：フォルダー 内のdbf ファイル の LDID と CPGファイルの情報を一覧で表示

※国土数値情報の土地利用3次メッシュ（平成28年）のデータをサンプルとして利用しています。

<pre>
folder
  |- xxx.dbf
  |- (xxx.cpg)
</pre>

In [3]:
folder = r"your_folder_path\国土数値情報DB\1_国土_土地利用\土地利用3次メッシュ\L03-a-16_6140-jgd_GML"

In [4]:
import glob
dbf_files = glob.glob(folder + "/*.dbf") # フルパスのリスト
files = []
for dbf in dbf_files:
    # LDIDと関連したCodepage の情報
    code = codepage_list_from_dbf_file(dbf) # dbf ヘッダーのLDID をもとにCodepage の情報を取得
    path, dbf_name = os.path.split(dbf) # path と ファイル名を分ける
    code.insert(0, dbf_name) # list の先頭にファイル名を追加

    # CPG ファイルの情報
    cpg_exist, code_value = check_related_cpgfile(dbf)
    code.append(cpg_exist)
    code.append(code_value)
    
    files.append(code)

In [5]:
# Pandas で表示
import pandas as pd
df = pd.DataFrame(files)
df.columns = ['ファイル名', 'LDID', 'Codepage', 'Description', 'CPGファイルの有無', 'CPGファイルの値'] # dataFrame のヘッダーを設定
df

,ファイル名,LDID,Codepage,Description,CPGファイルの有無,CPGファイルの値
0,L03-a-16_6140.dbf,0,0,Unknown CodePage,False,


#### 2) サンプルその２：サブフォルダー 内のdbf ファイル の LDID と CPGファイルの情報を一覧で表示

※国土数値情報の土地利用3次メッシュ（平成28年）のデータをサンプルとして利用しています。
<pre>
folder
  |- subfolder
    |- xxx.dbf
    |- (xxx.cpg)
  |- subfolder
    |- yyy.dbf
    |- (yyy.cpg)
</pre>

In [7]:
folder2 = r"your_folder_path\国土数値情報DB\1_国土_土地利用\土地利用3次メッシュ"

In [8]:
files2 = []
dbf_files = glob.glob(folder2 + "/**/*.dbf", recursive = True) # フルパスのリスト
for dbf in dbf_files:
    # LDIDと関連したCodepage の情報
    code = codepage_list_from_dbf_file(dbf) # dbf ヘッダーのLDID をもとにCodepage の情報を取得
    path, dbf_name = os.path.split(dbf) # path と ファイル名を分ける
    
    code.insert(0, os.path.split(path)[1]) # list の先頭にサブディレクトリを追加
    code.insert(1, dbf_name) # list の先頭にファイル名を追加
    
    # CPG ファイルの情報
    cpg_exist, code_value = check_related_cpgfile(dbf)
    code.append(cpg_exist)
    code.append(code_value)
    files2.append(code)

In [9]:
df2 = pd.DataFrame(files2)
df2.columns = ['サブフォルダー名','ファイル名', 'LDID', 'Codepage', 'Description', 'CPGファイルの有無', 'CPGファイルの値'] # dataFrame のヘッダーを設定
df2

,サブフォルダー名,ファイル名,LDID,Codepage,Description,CPGファイルの有無,CPGファイルの値
0,L03-a-16_6040-jgd_GML,L03-a-16_6040.dbf,0,0,Unknown CodePage,False,
1,L03-a-16_6041-jgd_GML,L03-a-16_6041.dbf,0,0,Unknown CodePage,False,
2,L03-a-16_6140-jgd_GML,L03-a-16_6140.dbf,0,0,Unknown CodePage,False,
3,L03-a-16_6141-jgd_GML,L03-a-16_6141.dbf,0,0,Unknown CodePage,False,


#### 3) サンプルその３：フォルダー 内のシェープファイル に対応するCPG ファイルを一括で作成する

サンプルその１ で調査したフォルダーに対応して、'シェープファイル名.cpg' のテキストファイルを一括で作成するサンプル。

dBASE ファイル の LDID が`13` でなく`0` となっている Shift-JIS シェープファイルが大量にある場合に便利です。

In [11]:
def create_related_cpgfile(cpg_txt, dbf):
    """
    dbf ファイルと同じフォルダー内にcpg ファイルを作成
    """
    path, dbf_name = os.path.split(dbf) # path と ファイル名を分ける  
    file_name = os.path.splitext(dbf_name)[0]
    cpg_file_name = "{}.cpg".format(os.path.splitext(file_name)[0])
    cpg_file = os.path.join(path, cpg_file_name) #フルパス
    with open(cpg_file, 'w', encoding='utf-8') as f: # mode='w' でない場合には作成。既存のものがある場合は上書き
        f.write(cpg_txt)
        
def delete_related_cpgfile(dbf):
    """
    dbf ファイルと同じフォルダー内にcpg ファイルを削除
    """
    path, dbf_name = os.path.split(dbf) # path と ファイル名を分ける  
    file_name = os.path.splitext(dbf_name)[0]
    cpg_file_name = "{}.cpg".format(os.path.splitext(file_name)[0])
    cpg_file = os.path.join(path, cpg_file_name) #フルパス
    if os.path.isfile(cpg_file): # 存在する場合に削除
        os.remove(cpg_file)

In [12]:
cpg_txt = 'SJIS\n' # 'SJIS\n' or 'UTF-8\n'
dbf_files = glob.glob(folder + "/*.dbf") # フルパスのリストを取得
for dbf in dbf_files:
    create_related_cpgfile(cpg_txt, dbf)
    #delete_related_cpgfile(dbf)

#### 4) サンプルその４：サブフォルダー 内のシェープファイル に対応するCPG ファイルを一括で作成する

サンプルその２ で調査したサブフォルダーに対応して、'シェープファイル名.cpg' のテキストファイルを一括で作成するサンプル。

In [13]:
cpg_txt = 'SJIS\n' # 'SJIS\n' or 'UTF-8\n'
dbf_files = glob.glob(folder2 + "/**/*.dbf", recursive = True) # フルパスのリストを取得
for dbf in dbf_files:
    create_related_cpgfile(cpg_txt, dbf)
    #delete_related_cpgfile(dbf)